Mount Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
ls

drive/  sample_data/


Installation of Required Packages

In [ ]:
!pip install lightning==2.4.0
# !pip install nemo_toolkit==2.0.0
!pip install transformers datasets sentencepiece librosa jiwer soundfile
# 1. Fix the NumPy version (Crucial for Colab right now)
!pip install "numpy<2.0"

# 2. Install missing NeMo dependency
!pip install hydra-core

# 3. Re-install NeMo with ASR dependencies just to be safe
!pip install "nemo_toolkit[asr]==2.0.0"

# Note: You MUST restart the runtime after running this cell for the NumPy change to take effect.
# Go to "Runtime" -> "Restart Session" (or Ctrl+M .)
# Force uninstall conflicting version and install standard version
!pip uninstall -y numpy
!pip install "numpy<2.0"
!pip install datasets==3.0.2
!pip install pyarrow==14.0.1
!pip install nemo_toolkit==2.0.0

In [ ]:
DATA_DIR = "Path to your Tokenizer Directory"
BASE_DIR = "Path to your base directory or Folder"
MANIFEST_DIR = f"{BASE_DIR}/manifests"
AUDIO_DIR = f"{BASE_DIR}/audio/decoded_audio"
TOKENIZER_DIR = f"{BASE_DIR}/tokenizer"
CHECKPOINT_DIR = f"{BASE_DIR}/checkpoints"
LOG_DIR = f"{BASE_DIR}/logs"

## BPE tokenizer


In [ ]:


import os
import json
import sentencepiece as spm
import nemo.collections.asr as nemo_asr
from nemo.collections.common.tokenizers import SentencePieceTokenizer

print("Starting BPE Tokenizer training...")

# DATA_DIR = "/content/drive/MyDrive/ASR_richard/tokenizer"
text_corpus_path = os.path.join(DATA_DIR, 'text_corpus.txt')

# --- Step 1: Extract text corpus ---
print("Extracting text from manifest...")
with open(os.path.join(MANIFEST_DIR, 'train_manifest.json'), 'r') as f_manifest, \
     open(text_corpus_path, 'w') as f_text:
    for line in f_manifest:
        try:
            entry = json.loads(line)
            f_text.write(entry['text'] + '\n')
        except json.JSONDecodeError:
            print(f"Skipping bad line: {line}")

print("✅ Text corpus extracted.")

# --- Step 2: Train SentencePiece tokenizer directly ---
print("Training SentencePiece model...")
tokenizer_prefix = os.path.join(DATA_DIR, 'tokenizer_1024_bpe')

spm.SentencePieceTrainer.Train(
    input=text_corpus_path,
    model_prefix=tokenizer_prefix,
    vocab_size=1024,
    model_type='bpe',
    character_coverage=1.0,
    bos_id=1,
    eos_id=2,
    unk_id=0,
    pad_id=-1,
    user_defined_symbols=[]
)

print(f"✅ SentencePiece model trained and saved to:")
print(f"{tokenizer_prefix}.model")
print(f"{tokenizer_prefix}.vocab")

# --- Step 3: Load into NeMo Tokenizer for verification ---
tokenizer = SentencePieceTokenizer(model_path=f"{tokenizer_prefix}.model")
print("✅ NeMo tokenizer loaded successfully!")
print("Sample tokens:", tokenizer.text_to_tokens("HELLO WORLD"))


## imports and all.........

In [ ]:
import huggingface_hub
import os
import json
import torch
# import lightning.pytorch as pl
# from lightning.pytorch.callbacks import ModelCheckpoint, EarlyStopping
# from lightning.pytorch.loggers import TensorBoardLogger
# from lightning.pytorch import Trainer
import glob
import os
import json
import sentencepiece as spm
print("huggingface_hub:", huggingface_hub.__version__)
from omegaconf import OmegaConf, open_dict


from nemo.collections.common.tokenizers import SentencePieceTokenizer
from nemo.collections.asr.models import EncDecCTCModel
import pytorch_lightning as pl
from pytorch_lightning.callbacks import ModelCheckpoint, EarlyStopping
from pytorch_lightning.loggers import TensorBoardLogger
from pytorch_lightning import Trainer

print("✅ NeMo imports working fine!")

## Model loading

In [ ]:
BASE_DIR = "/content/drive/MyDrive/ASR_richard"
MANIFEST_DIR = f"{BASE_DIR}/manifests"
AUDIO_DIR = f"{BASE_DIR}/audio/decoded_audio"
TOKENIZER_DIR = f"{BASE_DIR}/tokenizer_nemo"
CHECKPOINT_DIR = f"{BASE_DIR}/checkpoints"
LOG_DIR = f"{BASE_DIR}/logs"

os.makedirs(CHECKPOINT_DIR, exist_ok=True)
os.makedirs(LOG_DIR, exist_ok=True)

# --- 1. Use Your Defined Paths ---


TRAIN_MANIFEST = f"{MANIFEST_DIR}/train_manifest.json"
DEV_CLEAN_MANIFEST = f"{MANIFEST_DIR}/dev_clean_manifest.json"
DEV_OTHER_MANIFEST = f"{MANIFEST_DIR}/dev_other_manifest.json"

# You can Increase Batch size as per your GPU V-RAM Capacity
BATCH_SIZE = 40
NUM_WORKERS = 25
SAMPLE_RATE = 16000

# --- 2. Load the Pre-trained Conformer-CTC Model ---
print(f"\nLoading pre-trained model: ")

asr_model = EncDecCTCModel.from_pretrained("nvidia/stt_en_fastconformer_ctc_large")
print("✅ Model loaded.")



asr_model.change_vocabulary(
    new_tokenizer_dir=TOKENIZER_DIR,
    new_tokenizer_type="bpe"
)

print("✅ Tokenizer updated and Decoder resized.")

cfg = asr_model.cfg

with open_dict(cfg):
    # Stronger SpecAugment (robustness) to improve WER on dev_other
    cfg.spec_augment.freq_masks = 3
    cfg.spec_augment.freq_width = 30
    cfg.spec_augment.time_masks = 10
    cfg.spec_augment.time_width = 0.08

    # Lower LR for fine-tuning
    cfg.optim.sched.name = "NoamHoldAnnealing"
    cfg.optim.sched.warmup_steps = 0
    cfg.optim.sched.hold_steps = 1000000  
    cfg.optim.sched.decay_rate = 1.0
    cfg.optim.sched.min_lr = 5e-5




# --- 4. Setup Data Loaders (ONLY DO THIS NOW) ---
# We define the configs here and load them AFTER the vocab change.

train_cfg = {
    "manifest_filepath": TRAIN_MANIFEST,
    "sample_rate": SAMPLE_RATE,
    "batch_size": BATCH_SIZE,
    "shuffle": True,
    "num_workers": NUM_WORKERS,
    "pin_memory": True,
    "trim_silence": True,
    # "max_duration": 20.0,
    "min_duration": 0.1,
    "is_tarred": False,
    "persistent_workers": True
}

val_other_cfg = {
    "manifest_filepath": DEV_OTHER_MANIFEST,
    "sample_rate": SAMPLE_RATE,
    "batch_size": BATCH_SIZE,
    "shuffle": False,
    "num_workers": NUM_WORKERS,
    "name": "dev_other",
}

print("Setting up Training Data...")
asr_model.setup_training_data(train_cfg)

print("Setting up Validation Data...")
asr_model.setup_validation_data(val_other_cfg)

print("✅ Model successfully configured!")

In [ ]:
# Checkpoint Callback
checkpoint_callback = ModelCheckpoint(
    dirpath=CHECKPOINT_DIR,
    filename="fastconformer-{epoch:02d}-{val_wer:.3f}",
    monitor="val_wer",  
    mode="min",
    save_top_k=2,
    save_last=True,
    verbose=True
)


# Logger
logger = TensorBoardLogger(save_dir=LOG_DIR, name="FastConformer_Finetune")

In [ ]:
# --- 5. Set Up Trainer (Using your settings) ---
trainer = Trainer(
    accelerator="gpu",
    devices=1,
    # precision=16,               
    precision="bf16-mixed",                
    max_epochs=20,               
    logger=logger,
 
    callbacks=[checkpoint_callback],
    val_check_interval=1.0,
    accumulate_grad_batches=4,   # Gradient accumulation
    log_every_n_steps=100,
    default_root_dir=CHECKPOINT_DIR,
)

# Attach trainer (only need to do this once)
asr_model.set_trainer(trainer)
print("✅ Trainer initialized. Without Early Stopping")

In [ ]:
from omegaconf import open_dict

# 1. Modify the decoding strategy in the model config
with open_dict(asr_model.cfg.decoding):
    asr_model.cfg.decoding.strategy = "greedy_batch"

print(f"✅ Decoding strategy changed to: {asr_model.cfg.decoding.strategy}")

✅ Decoding strategy changed to: greedy_batch


In [ ]:
import torch
import omegaconf


#    and forces weights_only=False
def force_unsafe_load(f, map_location=None, pickle_module=None, **pickle_load_args):
    # Force the safe-load flag to False
    pickle_load_args['weights_only'] = False

    # Check if we've already backed up the original (to avoid recursion loops)
    if hasattr(torch, '_original_load'):
        return torch._original_load(f, map_location, pickle_module, **pickle_load_args)
    else:
        # Fallback if _original_load isn't set yet 
        return torch.jit.load(f) 

# 2. Apply the patch
if not hasattr(torch, '_original_load'):
    torch._original_load = torch.load  # Save the real function
    torch.load = force_unsafe_load     # Replace with forced wrapper

print("✅ Applied STRONG patch: 'weights_only=False' is now forced globally.")



✅ Applied STRONG patch: 'weights_only=False' is now forced globally.


In [ ]:
from nemo.collections.asr.models import EncDecCTCModelBPE

manual_ckpt = "/content/drive/MyDrive/ASR_richard/checkpoints/fastconformer-epoch=19-val_wer=0.061.ckpt"

print("🔄 Loading model weights ONLY (no decoder reset)")
asr_model = EncDecCTCModelBPE.load_from_checkpoint(manual_ckpt)

# ❌ DO NOT call change_vocabulary here
# ❌ DO NOT touch tokenizer again

# Apply SpecAugment + LR changes
cfg = asr_model.cfg
with open_dict(cfg):
    cfg.spec_augment.freq_masks = 3
    cfg.spec_augment.freq_width = 30
    cfg.spec_augment.time_masks = 10
    cfg.spec_augment.time_width = 0.08

    cfg.optim.lr = 5e-5

    # constant LR scheduler
    cfg.optim.sched.name = "NoamHoldAnnealing"
    cfg.optim.sched.warmup_steps = 0
    cfg.optim.sched.hold_steps = 1_000_000
    cfg.optim.sched.decay_rate = 1.0
    cfg.optim.sched.min_lr = 5e-5

# Reattach data
asr_model.setup_training_data(train_cfg)
asr_model.setup_validation_data(val_other_cfg)

asr_model.set_trainer(trainer)
trainer.fit(asr_model)


## Export / Save the Model

In [ ]:
best_ckpt_path = checkpoint_callback.best_model_path
print("Best checkpoint:", best_ckpt_path)

# Load PL checkpoint correctly
best_model = EncDecCTCModel.load_from_checkpoint(best_ckpt_path)

# Export to .nemo format
best_model.save_to(f"{CHECKPOINT_DIR}/best_model_04.nemo")

print("✅ Successfully exported best model to .nemo")


In [ ]:
asr_model.save_to(f"{CHECKPOINT_DIR}/fastconformer_finetuned_dev_other_03.nemo")
print("✅ Model fine-tuned and saved successfully!")

✅ Model fine-tuned and saved successfully!
